In [21]:
#Vector db is effectively a vector store with extra database features like
#clustering,scaling,security, backups
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
import ipykernel
import sentence_transformers


In [22]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

video_id = "pBRSZBtirAk"
try:
    # Create instance
    ytt = YouTubeTranscriptApi()
    
    # Fetch the transcript
    fetched = ytt.fetch(video_id, languages=["en"])
    
    # If you want list-of-dicts format:
    transcript_list = fetched.to_raw_data()
    
    # Join all text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)
    
except TranscriptsDisabled:
    print("NO CAPTIONS IN VIDEO")
except Exception as e:
    print("Error:", e)

this is a world where somebody can have over hundred billion dollar what is that these fortunes are almost illegitimate unless in a very smart way given back one of the biggest names in business Tech and philanthropy exploring and investing in innovative solutions to some of the world's toughest problems Bill Gates starting with Microsoft where I had monomaniacal Focus giving up weekends and vacation wasn't some big sacrifice I loved it the idea that everybody would use computer was somewhat ridiculed so it was kind of fun saying no no this is really going to be mainstream if you look at it from the outside you see Steve Jobs and yourself having this competition Envy jealousy what was it actually like he said we were the General Motors and he was Mercedes social networking we're still arguing about what the policy should be algorithms reward outrageous things even if they're not at all factual I've always underestimated how incredible my father was he would say hey I'm sorry I worked s

In [23]:
#Step 1 - Text Splitting

splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks= splitter.create_documents([transcript])
len(chunks)


90

In [24]:
chunks[0]

Document(metadata={}, page_content="this is a world where somebody can have over hundred billion dollar what is that these fortunes are almost illegitimate unless in a very smart way given back one of the biggest names in business Tech and philanthropy exploring and investing in innovative solutions to some of the world's toughest problems Bill Gates starting with Microsoft where I had monomaniacal Focus giving up weekends and vacation wasn't some big sacrifice I loved it the idea that everybody would use computer was somewhat ridiculed so it was kind of fun saying no no this is really going to be mainstream if you look at it from the outside you see Steve Jobs and yourself having this competition Envy jealousy what was it actually like he said we were the General Motors and he was Mercedes social networking we're still arguing about what the policy should be algorithms reward outrageous things even if they're not at all factual I've always underestimated how incredible my father was h

In [25]:
#Embedding and storing in vs

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store=FAISS.from_documents(chunks,embeddings)

In [26]:
vector_store.index_to_docstore_id

{0: '8d6d092c-1385-4a71-8d97-a4adea28b13c',
 1: 'b1b89f8b-4c19-4b1b-93d8-dad911e8ae56',
 2: '4d5f6573-cb3b-46fa-88a2-bb0c9cad89b4',
 3: 'e75e1d01-a839-4d2c-8cac-a8d3677e87b8',
 4: 'a16edb10-e807-4e6a-85dd-f43ac1041aea',
 5: '9cc01085-25df-4634-a4fd-614fae9305a9',
 6: '57b25edc-c80d-4394-8b2a-0e7d8c08252c',
 7: 'efdd670f-fcbe-44e2-8b6e-12117780d257',
 8: '00aff084-4cde-4371-820d-686094a57f23',
 9: 'f1e73161-8ec2-4340-b683-8fbdb8ff44ee',
 10: '54d8e6b2-239a-4772-9897-346489ec58ee',
 11: '582e221b-d06b-4b2b-9e12-321363f35d1a',
 12: '96b5fca8-23ed-4522-9a81-d35d56a1b869',
 13: 'aad755a0-5f44-4f2f-96c7-3c355bff8dfa',
 14: 'c70b2bb1-afa2-4371-8bcd-d0265dccfd5b',
 15: '17460cf9-e2e7-47ab-8efd-9f498a20fe1c',
 16: 'c8f91967-30f2-4ece-9c7a-fe8f0d1bb1db',
 17: 'fc28c0f2-32af-4db6-8280-3bf24e1ee2b7',
 18: 'b5c1db02-0dd5-426a-ad4f-0703cb20457a',
 19: 'e1e3f300-e3f1-4c81-9321-1c853ed426e7',
 20: '869ad8a3-4765-456d-aea9-7e0b88b227d3',
 21: '07637286-c890-4baf-a067-2d0fb48f8934',
 22: '39a64e89-5c9a-

In [27]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16a5defd0>, search_kwargs={'k': 5})

In [28]:
retriever.invoke('Facebook and X')

[Document(id='ee4716c6-1873-40f1-9871-c51a67e0bbc5', metadata={}, page_content="out uh so that scared me what is your favorite mental health habit this show is sponsored by better help we talk a lot about red flags in relationships the things to avoid but what about green flags those qualities that show someone is kind supportive and emotionally safe like when they listen with empathy respect your boundaries or celebrate your growth therapy helps you not only spot these green flags but also cultivate them in yourself and your relationships it's a space to learn grow and build connections rooted in trust and love whether you're dating married creating deeper friendships or focusing on your own Journey therapy can guide you toward relationships that pour positivity back into your life that's where better help comes in it's fully online making therapy convenient and affordable with over 5 million people already using it worldwide you'll have access to a diverse network of more than 30,000

In [29]:
#Step 2 - Augmentation (Query + Retrived Doc Merge then give it to LLM Step)
load_dotenv()
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
)


In [30]:
prompt = PromptTemplate(
    template="""
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [31]:
question= "is the topic of Facebook and X discussed in this video? if yes then what is discussed?"
retrieved_docs= retriever.invoke(question)

In [32]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"know personal computers then internet then cell phones uh and now ai but it's because it's it's super intelligence um it's of a different character and we'll it'll put us to the test on how we work together within the US and how how countries work together yeah it's uh interesting because as you talking about social media there I think Mark Zuckerberg on J Rogan was saying that they are taking away their fact Checkers and I believe ax is X is doing something similar what was your take on that you know this whole thing of how you balance Free Speech versus not you know discouraging people from using vaccines when that would be beneficial for them or even extreme stuff like Holocaust denial or some bad things there I'm a little bit disappointed that my generation hasn't got a clear prescription to how uh we achieve both the goals free speech and yet uh uh reasonable discourse that's not misleading people I mean during the pandemic you know the negativity about vaccines some of which you

In [33]:
final_prompt = prompt.invoke({"context": context_text, "question": question})
final_prompt

StringPromptValue(text="\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      know personal computers then internet then cell phones uh and now ai but it's because it's it's super intelligence um it's of a different character and we'll it'll put us to the test on how we work together within the US and how how countries work together yeah it's uh interesting because as you talking about social media there I think Mark Zuckerberg on J Rogan was saying that they are taking away their fact Checkers and I believe ax is X is doing something similar what was your take on that you know this whole thing of how you balance Free Speech versus not you know discouraging people from using vaccines when that would be beneficial for them or even extreme stuff like Holocaust denial or some bad things there I'm a little bit disappointed that my generation hasn't got a clear prescription to how uh we achieve both the goals free 

In [34]:
#Step 3 - Generation

model = ChatHuggingFace(llm=llm)

result = model.invoke(final_prompt)
print(result)

content=" \n    Reply: During the conversation, they briefly touched on the balancing act between free speech and combatting misinformation on social media platforms. Elon Musk was mentioned as a possible future philanthropist and the need for individuals to develop self-reflection and critical thinking skills in order to navigate through the vast amount of information available in the current age. There was no specific discussion of Facebook and X's approach to handling false information on their platforms. Musk's current relationship with other tech giants was also briefly mentioned with Musk stating that he may become more philanthropic as he ages. The topic of Bill Gates' focus on combatting diseases such as HIV and polio was brought up, with appreciation for Gates' continued efforts in this area." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 1048, 'total_tokens': 1209}, 'model_name': 'HuggingFaceH4/zephyr-7b-beta', 'system_fing